# Wolf and Chickens Game

Group formed by Gonzalo Marcos, Ali Ezzeddine and Isabel Alessandrini

                                                INTRODUCTION
                                        
This exercise aims to simulate the different dynamics involved in the interactions between a wolf, chickens and the food present in their environment. The different epucks of the simulation will represent the animals and each will have a series of behaviours and routines that will determine how and when they look for food, chase and run from eachother, and rest from their routine.

The environmental design and the cognitive modelling is based on the work of Braitenberg, V. "Vehicles: Experiments in Synthetic Psychology. Where "animal brains seem to be interpretable as pieces of computer machinery because of their simplicity and/or regularity" (Braitenberg, V., 1998). In robotics research and development, designers and engineers have for decades looked to biological systems for inspiration in building functional, flexible and robust robots (Bar-Cohen, 2005).

The scene is a farm in a wooded area with chickens running around looking for food. The scenario includes trees of different sizes and textures (different species), small hills (bump), plants, cups and walls to simulate the corral. There are also two farmers looking after the chickens.

The cognitive modelling simulate different behaviours and interactions between chikens and wolf:
Chickens (two):
- Obstacle avoidance (collision)
- Speed
- Hunger --> Food gathering --> Energy (speed level)
- Fear (Wolf)
- Love / Protection (Bill the farmer)


Wolf:
- Avoidance (collision)
- Speed
- Hunger --> Hunter --> Energy (Speed level)
- Fear (Bill the Farmer)

In the following coding sections, we will program the different behaviors of chickens and wolves using CoppeliaSim:


First, we will start loading the scene `epuck-scene-4_ChickenWolfandBills.ttt` provided along with this code, and we will start the simulator software.

In [ ]:
from simulator_interface import open_session, close_session
simulator, epuck1, epuck2, epuck3 = open_session(n_epucks=3)

We will also assign which role each epuck will have in the simulation.

In [ ]:
epuck1.species = "chicken"
epuck2.species = "chicken"
epuck3.species = "wolf"

Next we will start defining the different behaviours and routines that the chickens and wolf will have. The first one, obstacle avoidance, will be shared by all as they will want to avoid colliding with the different obstacles present in the environment.

In [ ]:
def obstacle_avoidance(robot):
    left, right = robot.prox_activations(tracked_objects=["20cm", "Tree", "Cup", "Bump"]) # If more obstacles are added, they will have to be identified here.
 
    return 1 - right, 1 - left

The chickens want to obtain food from the environment while simultaneously avoid the wolf to not get eaten. We will represent the food with a series of spheres that will appear throughout the environment. In order to generate them, we will run the following code:

In [ ]:
simulator.start_sphere_apparition(period=15, min_pos=[-1, -1, 1], max_pos=[3, 3, 2]) # The min_pos and max_pos arguments can be modified to restrict where the food will appear

Now that we have food, we will define the behaviours that will allow the chickens to interact with it. We will start with the hunger, a routine that will condition if the chicken wants to look for food. When the chicken is full, they are sated and walk slowly. Once they have digested the food, they will become more desperate and will run faster.

In [ ]:
def hunger(robot): 
    if robot.has_eaten():
        robot.hunger_level = 0  # if the chicken has eaten a sphere, it is no longer hungry and moves slower
        robot.max_speed = 6
    else:
        robot.hunger_level += 0.01  # otherwise (nothing eaten), it will get increasingly hungrier
    
    if robot.hunger_level > 0.7:
        robot.max_speed = 10 # When the chicken gets hungry enough, they start running faster

    robot.hunger_level = min(1., max(robot.hunger_level, 0.)) # We bound the hunger level between 0 and 1

With the previous rutine defined, we can now implement a food chasing behaviour that will be influenced by how hungry the chicken is.

In [ ]:
def food_chasing(robot):
    left, right = robot.prox_activations(tracked_objects=["Sphere"])

    return right, left, robot.hunger_level

The chickens can now avoid obstacles, feel hunger and search for food if they are huhgry enough, but we still have to make them fear the wolf and run away from it, so we will implement the wolf avoidance behaviour.

In [ ]:
def wolf_avoidance(robot):
    (left, right), (epuck_left, epuck_right) = robot.prox_activations(tracked_objects=["ePuck"], return_epucks=True)
    
    # left_species, right_species = robot.sensed_epuck_attributes(epuck_left, epuck_right, "species", default_value ="none") # Retrieve the species attributes of the sensed epucks

    # left_activation = left if left_species == "wolf" else 0
    # right_activation = right if right_species == "wolf" else 0  # We avoid the epuck sensed if it's a wolf
    
    # At first the script avoided only wolves, but after further consideration the chickens will avoid all ePucks, not only wolves
    # since they don't want to collide with eachoter and from a survival standpoint they want to spread so they are harder to catch and don't steal food from eachother
    
    return left, right

We will add a final behaviour to the chickens to make them smarter. When they are not hungry, they will try to hide with the farmers ("Bill") so they can avoid the wolf instead of running around. Once they get hungry enough, they will no longer want to hide as they will have to risk going outside for food.

In [ ]:
def hiding(robot):
    left, right = robot.prox_activations(tracked_objects=["Bill"])

    return right, left, 1 - robot.hunger_level # The less hungry, the more the chicken priorizes hiding themselves

Once the chicken behaviours are defined, we will record the internal variables of the chickens in order to plot them later.

In [ ]:
def chicken_log(robot):
    # Retrieve the values of the speed and hunger:
    robot.add_log("speed", robot.max_speed)
    robot.add_log("hunger", robot.hunger_level)

Now we begin with programming the behaviors and rootines of the wolf. First we begin with the chicken hunting behavior. This behavior will be controlled by an internal state of the wolf which is his hunger level. We will first define this rootine in the cell below. 

In [ ]:
def wolf_hunger(robot):
    left, right = robot.prox_activations(tracked_objects=["ePuck"])
    distance = (left + right)/2 
    if distance > 0.8: 
        robot.hunger_level = 0 # if the robot has eaten a chicken, decrease its hunger level to 0
    else:
        robot.hunger_level += 0.2  # otherwise (nothing eaten), increase the hunger level by 0.2
    # The line below bounds the value of the hunger level between 0 and 1
    robot.hunger_level = min(1., max(robot.hunger_level, 0.))

In [ ]:
def chicken_hunting(robot):
    left, right = robot.prox_activations(tracked_objects=["ePuck"])
    left_wheel = right
    right_wheel = left
    
    return left_wheel, right_wheel, robot.hunger_level * robot.energy_level 

When the wolf begins hunting for chickens, the behavior will be controlled by an internal state which we will call energy level. This energy level decreases over a certain period of time. It start with 1 and decreases by 0.2. 

In [ ]:
def energy_level(robot):
    left, right = robot.prox_activations(tracked_objects=["ePuck"])
    if left or right > 0:
        robot.energy_level -= 0.2
    else: robot.energy_level = 1
        
    robot.energy_level = min(1., max(robot.energy_level, 0.))
    

Below is the code to implement a fear of the farmer (base of the chicken) behavior to the wolf. This behavior makes sure that the wolf stays away from the farmer because he is armed and dangerous. The behavior has a higher weight than the hunting behavior. 

In [ ]:
def fear_farmer(robot):
    left, right = robot.prox_activations(tracked_objects=["Bill"])

    return left, right

Once we have defined all the behaviours of the chickens and the wolf, we will assign them and let them interact with eachother.

In [ ]:
# We initialize internal variables, in order to establish the initial behaviours
# when the simulation starts, and to make sure all code works

epuck1.max_speed = 10 # Chicken are slower than the wolf
epuck2.max_speed = 10 
epuck3.max_speed = 12

epuck1.hunger_level = 1 # The chickens will start the simulation hungry
epuck2.hunger_level = 1
epuck3.hunger_level = 1

epuck3.energy_level = 1 # The wolf starts the simulation well rested

In [ ]:
for e in simulator.robots:
    e.detach_all_behaviors()
    e.stop()
    e.attach_behavior(obstacle_avoidance, freq=10) # All will use obstacle avoidance

# Attaching behaviours and routines for the chickens
epuck1.attach_routine(hunger, freq=1)
epuck2.attach_routine(hunger, freq=1)
epuck1.attach_routine(chicken_log, freq=1)
epuck2.attach_routine(chicken_log, freq=1)
epuck1.attach_behavior(food_chasing, freq=10)
epuck2.attach_behavior(food_chasing, freq=10)
epuck1.attach_behavior(wolf_avoidance, freq=10)
epuck2.attach_behavior(wolf_avoidance, freq=10)
epuck1.attach_behavior(hiding, freq=10)
epuck2.attach_behavior(hiding, freq=10)

# Attaching behaviours and routines for the wolf
epuck3.attach_routine(wolf_hunger, freq=1)
epuck3.attach_behavior(chicken_hunting, freq=10)
epuck3.attach_behavior(fear_farmer, freq=10)

# Begin the simulation
for e in simulator.robots:
    e.start_all_routines()
    e.start_all_behaviors()

Now that the simulation is running, we can plot the internal variables of the wolf and chickens. The following code will plot the hunger level over time of the chickens.

Note: Please let the simulation run for a while so more data can be collected. Alternatively, execute the code multiple times over time to see how the graph changes according to the epuck state.

In [ ]:
%pylab inline

# Plot the hunger levels recorded by the chickens
plot(epuck1.get_log("hunger"))
plot(epuck2.get_log("hunger"))
legend(["Chicken 1", "Chicken 2"])

xlabel("Time")
ylabel("Hunger level")
title("Plot of hunger level against time")

Next we will plot the speed of the chickens over time.

In [ ]:
%pylab inline

# Plot the speed recorded by the chickens
plot(epuck1.get_log("speed"))
plot(epuck2.get_log("speed"))
legend(["Chicken 1", "Chicken 2"])

xlabel("Time")
ylabel("Speed")
title("Plot of speed against time")

To finish the simulation, execute the following line:

In [ ]:
close_session(simulator)

                                                    RESULTS

As we presented in the introduction and coding section, the chickens and the wolf interact with each other and with the environment. Showing the cognitive model that simulates the behaviors and interactions between the chickens and the wolf in the real world, as our scene represents the farm Chicke's corral.

Cognitive Models and Interactions

Interaction with the Enviroment

First, they avoid obstacles of the environment through the method prox_activations. This method retrieves the activation levels of epuck's proximity sensors with respect to tracked objects, specified as a list containing the strings the walls "20cm", "tree", "cup", and "bump".The method returns the activation levels of sensors corresponding to the specified tracked objects. 

The returned activation levels are then unpacked into two variables; left and right. These are the two proximity sensors, one on the left side and one on the right side, and the activation levels of these sensors are retrieved.

Finally, the function computes and returns a tuple containing the obstacle avoidance outputs for the left and right sides of the robot. It does this by subtracting the activation levels from 1. This implies that the higher the activation level of a sensor, the closer an obstacle is. Subtracting from 1 inverts this relationship, so that higher values represent more open space. Therefore, 1 - right represents the avoidance level for obstacles on the right side, and 1 - left represents the avoidance level for obstacles on the left side. These values are then returned by the function. 

PICTURE 1 (avoiding collition)

Chickens Cognitive Model 

Both chickens have the same behaviors between them and different interactions for the environment, the wolf, and Bill the farmer, in total five behavior are described to provide a cognitive model of the "chicken" simulation.

The first behavior is the avoidance collection with the objects in the environment, explained above.

Second, hunger. The function "def hunger(robot)" simulates hunger behavior of chikens, adjusting the chicken's speed and hunger level based on whether it has eaten and how hungry it is. The If statement checks if the robot has eaten. The condition robot.has_eaten() is a method call that returns True if the robot has eaten and False otherwise.

If the chicken has eaten the balls that represents food (the condition is True), the chicken's hunger level is set to 0, indicating that it's not hungry anymore. Additionally, the maximum speed of the robot is set to 6 if the chicken hasn't eaten anything (False case of the if statement), the code inside the Else block is executed.

The chicken's hunger level is incremented by 0.01. This means that if the chicken hasn't eaten anything, its hunger level will gradually increase. Additionally, if the hunger level of the robot exceeds 0.7, the maximum speed of the robot is set to 10. This suggests that when the robot gets hungry enough, it will start moving faster.

Finally, the hunger level of the epuck is set between 0 and 1. This ensures that the hunger level doesn't exceed 1 or go below 0, preventing it from reaching unrealistic values.

PLOTS (maybe)

Third, the "food chasing" function allows the chickens to chase food represented by spheres using proximity sensors to detect their presence and proximity, while also providing information about the robot's hunger level for decision making purposes.

The method prox_activations retrieves the activation levels of epuck's proximity sensors, specifically looking for spheres in its vicinity. The method returns the activation levels of the left and right sensors, and the variables left and right are then assigned the activation levels of the sensors on the left and right sides of epuck, respectively. 

The return right, left, robot.hunger_level function returns a tuple containing three values; right and left (the activation levels of the proximity sensors in both directions) and the hunger level of the chicken with the decision making process in chasing food.

PICTURE 2 (chicken's eating) 

Four, wolf and chickens avoidance. This function allows the chickens to avoid colliding with and among themselves by using proximity sensors to detect the presence of the "wolf" and adjust their movement accordingly. The avoidance behavior helps ensure the survival and well-being of the chickens in the environment.

Similar to the avoidance objects, the "prox_activations" method returns the activation levels of the puck's proximity sensors for "wolf" and "chickens" in its vicinity to avoid collisions and ensure the chickens survival.

The returned values are unpacked into two tuples (left, right) containing the activation levels of the puck's proximity sensors in those directions on the left and right sides.

Fifth, the "Hide" function allows the chickens to prioritize hiding when they are full of food and are slower by being attracted to the "Bill" object. It uses proximity sensors to detect its presence and adjusts its behavior based on its hunger level. The less hungry the chickens are, the more they prioritize hiding.

Using the robot.prox_activations method, the epucks tracked "Bill", specifically looking for the object in its vicinity. Contrary to the avoidance behavior, the epuck's proximity sensors on the left and right side, and the hunger level, prioritize the chickens hiding themselves, calculated based on its hunger level. The hunger level is subtracted from 1, i.e. the less hungry the chickens are, the higher the priority of hiding over other behaviors.

PICTURE 3 (chickens hiding)

Wolf Cognitive Model

The wolf cognitive model includes 4 behaviors that we will discuss next.

The first behavior is the avoidance collection with the objects in the environment, explained above.

Second behavoir, "hunter". The wolf hunter behavior function controls the hunger behavior of the epuck "wolf", adjusting its hunger level based on whether it has eaten a chicken or not, as determined by the proximity of epuck objects (chickens).

The prox_activations method retrieves the activation levels of proximity sensors in epuck "wolf", specifically looking for epuck "chickens" in its vicinity. The returned values are assigned to the left and right variables, representing the activation levels of the robot's proximity sensors on the left and right sides, respectively. But this time the distance = (left + right)/2 calculates the average distance from the wolf robot to the nearest ePuck objects (chickens). If the distance is greater than 0.8, it means that the wolf robot has eaten a chicken. In this case, the wolf robot's hunger level is reduced to 0, indicating that it's no longer hungry. Otherwise, if the condition is not met (i.e. the distance is less than or equal to 0.8), the hunger level of the wolf robot increases by 0.2.

The hunger level of the wolf robot is bounded between 0 and 1. It uses the min() and max() functions to ensure that the hunger level doesn't exceed 1 or go below 0, preventing it from reaching unrealistic values.

PICTURE (Wolf eating a chicken)

Third, the "energy level" function simulates the energy control behavior of hunting, adjusting it based on the proximity of epuck objects "chickens" detected by its proximity sensors. The energy level decreases over time because the wolf gets tired of chasing chickens. 

The method "prox_activations" retrieves the activation levels of the proximity sensors, specifically looking for epuck objects "chickens". The activation levels are assigned to variables left and right of the proximity sensors.

If either left or right is greater than 0, it means that the "wolf" is close to an epuck "chicken". In this case the energy level of the robot is reduced by 0.2.  This behavior simulates the energy expenditure of the robot when it encounters the epuck "chickens". Conversely, if the condition is not met (i.e. both left and right are 0, indicating that there are no epuck objects nearby), the energy level of the "wolf" is set to 1, indicating that the robot does not expend energy when there are no epuck "chickens" nearby.

The energy level of the robot is bounded between 0 and 1, using the min() and max() functions to ensure that the energy level doesn't exceed 1 or go below 0, preventing it from reaching unrealistic values.

Four, fear behavior. The "Fear Farmer" function allows the epuck "Wolf" to sense the presence of an object named "Bill" using its proximity sensors, and it returns the activation levels of these sensors for further decision making or control within the simulated environment.

As we have already seen, the prox_activations method activates the levels of the proximity sensors specifically for "Bill" in its vicinity. The left and right sensors of the "wolf" epuck, respectively, indicate the presence and proximity of objects "Bill" in those directions and its avoidance.

Chickens and Wolf Interaction 

First, the speed of "chicken" and "wolf" is set to simulate real life in nature, where "chicken" is slower than "wolf". The "chicken" has a maximum speed of 10, while the "wolf" has a speed of 12. The three epucks start with the same speed of 1. Also, as we discussed in the section above, each cognitive model of "wolf" and "chickens" has its own speed control behavior related to hunger and/or energy level.

Then, to set up the behavior and interaction between the multiple epucks and simulate the behavior of "wolf" and "chickens", it was necessary to configure their routines behavior according to their roles.

The chicken's routines are attached with a frequency of 1, which means that they are executed in every simulation step. The behaviors such as "chasing food", "wolf avoidance" and "hiding" are attached with a frequency of 10, meaning they will be executed every 10 simulation steps.

Meanwhile, the "wolf" routines for "hunger" are attached with a frequency of 1, meaning they will be executed every simulation step. And the "chicken hunt" and "fear farmer" behaviors are attached with a frequency of 10, meaning they will be executed every 10 simulation steps.

In conclusion, we can say that each of the three epuck robots simulates an animal role with a cognitive model according to "chicken" and "wolf". Each role has its own behaviors determined by some internal control variables and/or its interaction with the environment and among each other. As in real life in "nature", animals perceive their enviroment with their bodies through stimuli (in our case through sensors) and act according to these feelings (fear, attraction, avoidance, chaining, hunting, etc.) and their own internal needs like hunger. But also animal bodies have some control behavior determined by its body structures (that in our case are determined by the code and simulating nature) that includes the speed and energy level (by tiring or hungry) that we concired in our project.

In the following section we will discuss the implications of this model and the related bibliography.



                                                DISCUSSION
